In [1]:
# 나의 구글 드라이브를 구글 본사 서버와 연결하는 작업
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# 현재 실행 경로 확인하기
import os
print(os.getcwd())

/content


In [3]:
# 현재 작업 폴더 nlp로 경로 이동하기
%cd '/content/drive/My Drive/nlp' 

/content/drive/My Drive/nlp


In [4]:
print(os.getcwd())

/content/drive/My Drive/nlp


In [5]:
#필요한 라이브러리 임폴트
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
"""
KoNLpy(코엔엘파이) 라이브러리 설치
KoNLpy(코엔엘파이) 라이브러리 설치 : 한국어 형태소 분석기로서, 한글 자연어 처리를 위해
만들어진 라이브러리
"""

# KoNLpy 라이브러리 설치
!pip install konlpy

In [7]:
# 필요한 라이브러리 임폴트
import konlpy
from konlpy.tag import Kkma, Okt
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [8]:
# 한글 형태소 분석기 생성

# kkma = Kkma()
okt = Okt()

In [9]:
# pandas.read_csv() 함수를 사용하여 데이터프레임 생성하기
import pandas as pd
df_data = pd.read_csv('titlelist.csv',encoding='utf-8', header=0)
# pd.options.display.max_columns = None

In [10]:
print(df_data)

                                             title
0      (주)광주글로벌모터스 AI역량검사 지원부스 운영<7.26(월)~7.27(화)>
1    (주)광주글로벌모터스 채용설명 취업특강<7.23(금), 7.26(월)> 운영 안내
2                    광주지방고용노동청 기업지원과 카카오톡 채널 개설 안내
3            구인구직만남의날 안내['21. 7. 8(목) 14시]-(주)진양산업
4               2021년도 광주지방고용노동청 채용박람회 운영 기관 모집 공고
..                                             ...
905                    구인구직 만남의 날(빛고을 job마당)(9.23)
906                 구인구직 만남의 날(빛고을 job마당)(9.22, 목)
907                      광주광역시 사회적기업 통합브랜드 명 공모 안내
908                       중소기업 구인구직 만남의 날(9.20, 화)
909                       중소기업 구인구직 만남의 날(9.16, 금)

[910 rows x 1 columns]


In [11]:
print(df_data.columns)

Index(['title'], dtype='object')


In [12]:
sentences = []

for sentence in df_data['title']:
  sentences.append(sentence)

# 결과 확인
print(len(sentences))

910


In [13]:
# key word 설정
'''
key word 설명
1.교육 = education --> key_words[0]
2.고용 = employ --> key_words[1]
3.주거 = dwelling --> key_words[2]
4.건강 = health --> key_words[3]
5.금융 = finance --> key_words[4]
6.문화 = culture --> key_words[5]

# 문장

'''

key_words = ['education', 'employ', 'dwelling', 'health', 'finance', 'culture']

key_words[0] = ['교육', '동아리', '수선방', '사전', '고졸', '훈련']
key_words[1] = ['고용','청년', '사업', '일자리', '대행자','채용','글로벌','취업','인력','직원','빅데이터','창업','비지니스']
key_words[2] = ['주거','투자','시행','비행']
key_words[3] = ['건강','백신','접종','클리닉','예방접종','코로나','임산부','의료']
key_words[4] = ['금융','디딤돌','서민','서민금융']
key_words[5] = ['문화','이야기','놀이','아름','동화','부엌','요리','공방','공모','친화','소통','비엔날렌','콘서트','창작','콘텐츠','영상','전시','다목적홀','김대중','홍보','행사',
                '디자인','그림','도서관','연극','공연','예술','판소리','국악','음악','체험','참여','풍류','무료','직장맘','국민']

education = []
employ = []
dwelling = []
health = []
finance = []
culture = []



In [ ]:
test=0

In [15]:
# 가져온 데이터 분류하기 - [1-1]
def classify(data=sentences):
  for sentence in data:
    list_sentence = okt.morphs(sentence)
    if len(set(list_sentence) & set(key_words[0])) > 0:
      education.append(sentence)
    elif len(set(list_sentence) & set(key_words[1])) > 0:
      employ.append(sentence)
    elif len(set(list_sentence) & set(key_words[2])) > 0:
      dwelling.append(sentence)
    elif len(set(list_sentence) & set(key_words[3])) > 0:
      health.append(sentence)
    elif len(set(list_sentence) & set(key_words[4])) > 0:
      finance.append(sentence)
    elif len(set(list_sentence) & set(key_words[5])) > 0:
      culture.append(sentence)  

In [16]:
# 문장을 분류하는 함수 실행 - [1-2]
classify(sentences)

In [17]:
# 문장을 분류하는 함수 실행의 결과 확인 - [1-3]
print("education = ", education) 
print("employ = ", employ) 
print("dwelling = ", dwelling) 
print("health = ", health) 
print("finance = ", finance) 
print("culture = ", culture) 

education =  ['[공지] 실업급여 교육시간 변경안내', '[공지] 실업급여 집체교육 방식 변경 안내', '[공지] 실업급여 관련 교육시간 안내', '[공지] 실업급여 교육시간 변경 안내', '[공지] 실업급여 교육시간 변경 안내', '실업자훈련 훈련비 자비부담액 환급 신청 안내', '[공지] 실업급여 교육시간 변경 안내', '직업능력개발훈련교사 자격 취소 공시송달 공고', '2018년 지역경제 위기 지역 소상공인 대상 전문기술교육 추가 지원 공고', '포스코 전기정비 외주파트너사 취업희망자 교육생 모집', '자비부담 훈련비 환급신청 안내', '♡ 포스코 외주파트너사(기계부분) 취업희망자 교육생 모집 ♡', '광주경영자총협회 『생산,품질경영시스템관리과정 2기』교육생 모집', '전자 회로설계(SMPS) 전문인력 양성과정 훈련생 모집 (전자부품연구원)', '일가정양립 가정의 달 행사 사전 이벤트 [가족함께 사진 공모전]', '일가정 양립을 위한 가정의 달 행사 사전이벤트<일가정양립 슬로건 공모>', '(에어)가전산업 재직자 대상 역량강화 기술교육(전자부품연구원)', '중소기업 멀티인재(3D설계,공정관리) 취업교육생 모집(조선대)', '자동차 애프터마켓 부품개발 전문인력양성과정, 튜닝 전문인력양성과정 교육생 모집(송원대)', '자동차 정비인력 교육과정 교육생모집 (송원대)', '중소기업 멀티인재(3D설계·공정관리) 취업교육생 모집(조선대학교 산학협력단)', '가전 R&D 연구인력 양성과정(채용연계) 교육생 모집 (전자부품연구원)', '생산,품질경영시스템관리과정 1기 훈련생 모집(광주경영자총협회)', '자동차 정비인력 교육과정 교육생모집(송원대학교)', '"자동차 전장부품 인력양성을 위한 2G-2C Frontier 프로젝트 사업" 교육생 모집', '『워크넷 사용자 교육』실시', '(직업훈련수료자)구인구직 만남의날-11.9(수)', '훈련과정별 취업성과 안내', '광주혁신프로젝트 세부내용(훈련, 고용서비스), 지역고용전문가 아카데미 신청 알림', '2016년 

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [19]:
#유사도 측정값

sent = education
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(sent)

In [20]:
idf = tfidf_vectorizer.idf_

In [21]:
print(dict(zip(tfidf_vectorizer.get_feature_names(),idf)))

{'11': 4.637586159726386, '12년': 4.637586159726386, '1기': 4.637586159726386, '2013년도': 4.232121051618222, '2014': 4.637586159726386, '2014년': 4.637586159726386, '2015': 4.637586159726386, '2015년': 4.637586159726386, '2016년': 3.028148247292285, '2016년도': 4.637586159726386, '2018년': 4.637586159726386, '2c': 4.637586159726386, '2g': 4.637586159726386, '2기': 4.637586159726386, '31기준': 4.637586159726386, '3d설계': 4.232121051618222, '4h': 4.637586159726386, 'frontier': 4.637586159726386, 'glp': 4.637586159726386, 'hrd센터': 4.637586159726386, 'it': 4.637586159726386, 'net': 4.637586159726386, 'oil': 4.637586159726386, 'smps': 4.232121051618222, 'work': 4.637586159726386, '가전': 4.637586159726386, '가전산업': 4.232121051618222, '가정의': 4.232121051618222, '가족함께': 4.637586159726386, '개인훈련제도': 4.637586159726386, '건강': 4.637586159726386, '경력단절': 4.637586159726386, '경영지원': 4.637586159726386, '고용노동부': 4.637586159726386, '고용서비스': 4.637586159726386, '공개강좌': 4.637586159726386, '공고': 3.7212954278522306, '공동': 4

In [15]:
education

NameError: ignored

In [ ]:

#데이터 넣어서 분류하기 
result = 0
end = 1

while end == 1:
    text2 = input("입력해주세요 : ")   
    if text2 ==0:
        break;
    text = okt.morphs(text2) 
    classify(sentences)
    if set(text) & set(key_words[0]) : 
      result = "교육"

    elif set(text) & set(key_words[1]) : 
      result = "고용"

    elif set(text) & set(key_words[2]) : 
      result = "주거"

    elif set(text) & set(key_words[3]) : 
      result = "건강"

    elif set(text) & set(key_words[4]) : 
      result = "금융"

    else:
      result = "문화"
    print("\n")  
    print("입력하신 테마는",result,"입니다.")
    print("\n") 





입력해주세요 : 고용


입력하신 테마는 고용 입니다.




KeyboardInterrupt: ignored

In [ ]:
#일단 bleu 공부

In [ ]:
from collections import Counter
import numpy as np
from nltk import ngrams


In [ ]:
# 단순 카운트 함수
def simple_count(tokens, 1): # 토큰화 된 candidate 문장, n-gram에서의 n 이 두 가지를 인자로 받음.
    return Counter(ngrams(tokens, 1)) #문장에서 n-gram을 카운트


In [ ]:
from collections import Counter
import numpy as np
from nltk import ngrams

# 단순 카운트 함수
def simple_count(tokens, n): # 토큰화 된 candidate 문장, n-gram에서의 n 이 두 가지를 인자로 받음.
    return Counter(ngrams(tokens, n)) #문장에서 n-gram을 카운트

candidate = "It is a guide to action which ensures that the military always obeys the commands of the party."
tokens = candidate.split() #단어 토큰화
result = simple_count(tokens, 1) #토큰화 된 문장, 유니그램의 개수를 구하고자 한다면 n=1
print(result)

Counter({('the',): 3, ('It',): 1, ('is',): 1, ('a',): 1, ('guide',): 1, ('to',): 1, ('action',): 1, ('which',): 1,
         ('ensures',): 1, ('that',): 1, ('military',): 1, ('always',): 1, ('obeys',): 1, ('commands',): 1, ('of',): 1, ('party.',): 1})

candidate = 'the the the the the the the'
tokens = candidate.split() #단어 토큰화
result = simple_count(tokens, 1)
print(result)

Counter({('the',): 7})

def count_clip(candidate, reference_list, n):
    cnt_ca = simple_count(candidate, n)
    # Ca 문장에서 n-gram 카운트
    temp = dict()

    for ref in reference_list: # 다수의 Ref 문장에 대해서 이하 반복
        cnt_ref = simple_count(ref, n)
        # Ref 문장에서 n-gram 카운트

        for n_gram in cnt_ref: # 모든 Ref에 대해서 비교하여 특정 n-gram이 하나의 Ref에 가장 많이 등장한 횟수를 저장
            if n_gram in temp:
                temp[n_gram] = max(cnt_ref[n_gram], temp[n_gram]) # max_ref_count
            else:
                temp[n_gram] = cnt_ref[n_gram]

    return {
        n_gram: min(cnt_ca.get(n_gram, 0), temp.get(n_gram, 0)) for n_gram in cnt_ca
        # count_clip=min(count, max_ref_count)
        # 위의 get은 찾고자 하는 n-gram이 없으면 0을 반환한다.
     }

candidate = 'the the the the the the the'
references = [
    'the cat is on the mat',
    'there is a cat on the mat'
]
result = count_clip(candidate.split(),list(map(lambda ref: ref.split(), references)),1)
print(result)

{('the',): 2}

def modified_precision(candidate, reference_list, n):
    clip = count_clip(candidate, reference_list, n) 
    total_clip = sum(clip.values()) # 분자

    ct = simple_count(candidate, n)
    total_ct = sum(ct.values()) #분모

    if total_ct==0: # n-gram의 n이 커졌을 때 분모가 0이 되는 것을 방지
      total_ct=1

    return (total_clip / total_ct) # 보정된 정밀도
    # count_clip의 합을 분자로 하고 단순 count의 합을 분모로 하면 보정된 정밀도

result=modified_precision(candidate.split(),list(map(lambda ref: ref.split(), references)),1) # 유니그램이므로 n=1
print(result)

def closest_ref_length(candidate, reference_list): # Ca 길이와 가장 근접한 Ref의 길이를 리턴하는 함수
    ca_len = len(candidate) # ca 길이
    ref_lens = (len(ref) for ref in reference_list) # Ref들의 길이
    closest_ref_len = min(ref_lens, key=lambda ref_len: (abs(ref_len - ca_len), ref_len))
    # 길이 차이를 최소화하는 Ref를 찾아서 Ref의 길이를 리턴
    return closest_ref_len

def brevity_penalty(candidate, reference_list):
    ca_len = len(candidate)
    ref_len = closest_ref_length(candidate, reference_list)

    if ca_len > ref_len:
        return 1
    elif ca_len == 0 :
    # candidate가 비어있다면 BP = 0 → BLEU = 0.0
        return 0
    else:
        return np.exp(1 - ref_len/ca_len)

def bleu_score(candidate, reference_list, weights=[0.25, 0.25, 0.25, 0.25]):
    bp = brevity_penalty(candidate, reference_list) # 브레버티 패널티, BP

    p_n = [modified_precision(candidate, reference_list, n=n) for n, _ in enumerate(weights,start=1)] 
    #p1, p2, p3, ..., pn
    score = np.sum([w_i * np.log(p_i) if p_i != 0 else 0 for w_i, p_i in zip(weights, p_n)])
    return bp * np.exp(score)

import nltk.translate.bleu_score as bleu


candidate = 'It is a guide to action which ensures that the military always obeys the commands of the party'
references = [
    'It is a guide to action that ensures that the military will forever heed Party commands',
    'It is the guiding principle which guarantees the military forces always being under the command of the Party',
    'It is the practical guide for the army always to heed the directions of the party'
]

print(bleu_score(candidate.split(),list(map(lambda ref: ref.split(), references))))
# 이번 챕터에서 구현한 코드로 계산한 BLEU 점수
print(bleu.sentence_bleu(list(map(lambda ref: ref.split(), references)),candidate.split()))
# NLTK 패키지 구현되어져 있는 코드로 계산한 BLEU 점수

Counter({('the',): 3, ('It',): 1, ('is',): 1, ('a',): 1, ('guide',): 1, ('to',): 1, ('action',): 1, ('which',): 1, ('ensures',): 1, ('that',): 1, ('military',): 1, ('always',): 1, ('obeys',): 1, ('commands',): 1, ('of',): 1, ('party.',): 1})
Counter({('the',): 7})
{('the',): 2}
0.2857142857142857
0.5045666840058485
0.5045666840058485
